In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable
from pyspark.sql.functions import current_date

In [0]:
df_faturas = (
    spark.read.table("dev.transient.faturas")
    .select(
        F.col("fatura_id"),
        F.col("reserva_id"),
        F.col("data_cobranca"),
        F.col("valor_total"),
        F.col("impostos"),
        F.col("forma_pagamento"),
        F.col("status_pagamento"),
        F.col("notas_fiscais")        
    )
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast("date"))
)

display(df_faturas.limit(5))


In [0]:
catalog_table = "dev.raw.tb_faturas"

try:
    delta_table = DeltaTable.forName(spark, catalog_table)

    delta_table.alias("target").merge(
        df_faturas.alias("source"),
        "target.reserva_id = source.reserva_id"
    ).whenMatchedUpdate(
        set={
            **{col: f"source.{col}" for col in df_faturas.columns if col != "update_date"},
            "update_date": current_date()
        }
    ).whenNotMatchedInsertAll().execute()

except Exception:
    df_faturas.write.format("delta").mode("overwrite").saveAsTable(catalog_table)